## Calculating likelihoods

In [1]:
import matplotlib.pyplot as plt
import corner as corner
import pandas as pd
import numpy as np
import h5py
import torch

from populations.bbh_models import get_models
import populations.bbh_models as read_models
from populations.utils.flow import NFlow
from populations.Flowsclass_dev import FlowModel
from populations import gw_obs

glasflow is using its own internal version of nflows


PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
#reading the pop synth file functions
def get_model_keys(path):
    alpha_val = '10'
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    f.close()

    # use only models with given alpha value
    for model in all_models:
        if 'alpha' in model:
            if 'alpha'+alpha_val in model:
                models.append('/'+model)
        else:
            models.append('/' + model)
    return(np.split(np.array(models), 5))

def get_model_keys_CE(path):
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    return(np.split(np.array(models), 4))

def read_hdf5(path, all_alpha=False):
    if all_alpha:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys_CE(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])
    else:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])

    return(popsynth_outputs)

In [3]:

params = ['mchirp','q', 'chieff', 'z']
param_bounds = np.array([[0,100], [0,1], [-1,1], [0,10]])
"""obs = np.random.rand(100,46,4)
obs[:,:,0] *=param_bounds[0,1]
obs[:,:,1] *=param_bounds[1,1]
obs[:,:,2] -= 0.5
obs[:,:,2] *=param_bounds[2,1]*2
obs[:,:,3] *=param_bounds[3,1]"""

'obs = np.random.rand(100,46,4)\nobs[:,:,0] *=param_bounds[0,1]\nobs[:,:,1] *=param_bounds[1,1]\nobs[:,:,2] -= 0.5\nobs[:,:,2] *=param_bounds[2,1]*2\nobs[:,:,3] *=param_bounds[3,1]'

In [4]:
file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
gw_path = '/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/gw_events'
observations, obsdata, p_theta, events = gw_obs.generate_observations(params, gw_path, \
                                            100, 'posteriors', None)

popsynth_outputs = read_hdf5(file_path, True)
flow = FlowModel.from_samples('CE', popsynth_outputs, params)
flow.load_model('/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/flow_models/', 'CE')

In [11]:
likelihoods = flow(obsdata,[0,1])

(46, 100, 4)
(46, 100, 4)
(46, 100)


In [12]:
print(likelihoods)

[6.60597305e-01 9.16298964e-02 9.90929334e-02 1.69736949e-01
 1.82608948e+00 9.45666301e-02 1.12873732e-01 2.17391304e+00
 2.00000005e+00 5.83961924e+00 5.19426177e-01 2.90160875e-01
 1.76031913e+00 1.88963299e-02 2.24207219e-01 1.40553681e-02
 8.55617637e+00 1.15680260e+00 4.13015485e-01 1.82665704e-01
 7.95931797e-02 1.57647617e+00 7.81326177e+00 2.64489054e+00
 2.12408962e+00 1.99639153e+00 5.62378744e-01 1.12325693e+00
 6.20778301e-01 1.93235872e+00 2.59999542e-01 3.84047012e-01
 1.38971383e-01 1.41022945e+00 1.72365618e-01 2.70470812e-01
 1.43294777e-01 1.76086972e+00 1.19361830e+00 2.17391304e+00
 5.02926987e-01 6.09310640e-01 3.44531316e-01 1.25363955e-06
 5.09206629e-01 8.98295514e-02]
